> We'll be implementing a classical dynamic programming algorithms to figure out the best action to take in a toy problem environment called slippery frozen lake 👩 

# Slippery Frozen Lake Environment

[Modified version of the description from Open AI Gym](https://gym.openai.com/envs/FrozenLake-v0/)

> The agent controls the movement of a character in a grid world. Some tiles of the grid are walkable, and others lead to the agent falling into holes with exploding 💥 bombs 💥 and die (I know right!) . Additionally, the movement direction of the agent is uncertain and only partially depends on the chosen direction. The agent is rewarded for finding a walkable path to a goal tile.

> You want to get to the target 🎯. The lake is mostly frozen, but there are a few holes where the ice has melted. If you step into one of those holes, you'll fall into the freezing water (where there is a BOMB 💥and you will EXPLODE 💥and DIE 💥). You navigate across the lake and go to the target 🎯. **However, the ice is slippery, so you won't always move in the direction you intend.**

> The episode ends when you reach the goal or fall in the hole with bomb and die!. You receive a reward of **+1** if you reach the goal 🎯, and **zero** otherwise.

# STATES

```
S - 👩🏼 (S: starting point, safe, REWARD = 0)
F - ▫️ (F: frozen surface, safe, REWARD = 0)
H - 💥 (H: hole with bomb, fall to your doom, terminal state, REWARD = 0)
G - 🎯 (G: goal, dartboard target, safe, terminal state, REWARD = +1) 

👩▫️▫️▫️ | SFFF 
▫️💥▫️💥 | FHFH 
▫️▫️▫️💥 | FFFH
💥▫️▫️🎯 | HFFG
```

In [1]:
from frozen_lake import SlipperyFrozenLake, FrozenLakeState, a_few_tests
from pprint import pprint

In [2]:
a_few_tests()

PASSED! :) 


In [3]:
frozen_lake_map = [
    ['S', 'F', 'F', 'F'], 
    ['F', 'H', 'F', 'H'],
    ['F', 'F', 'F', 'H'],
    ['H', 'F', 'F', 'G']]

lake_environment = SlipperyFrozenLake(frozen_lake_map)

# Explore Frozen Lake Environment

## chars ( S, F, H, G) - state condition

```
  0   1   2   3
  +---+---+---+---+
0 | S | F | F | F |
  +---+---+---+---+
1 | F | H | F | H |
  +---+---+---+---+
2 | F | F | F | H |
  +---+---+---+---+
3 | H | F | F | G |
  +---+---+---+---+
```

## icons ( 👩 ▫️  💥🎯)

```

  0   1   2   3
  +---+---+---+---+
0 |👩 |▫️ |▫️ |▫️|
  +---+---+---+---+
1 |▫️ |💥 |▫️ |💥|
  +---+---+---+---+
2 |▫️ |▫️ |▫️ |💥|
  +---+---+---+---+
3 |💥 |▫️ |▫️ |🎯|
  +---+---+---+---+

```

## terminal ( y / n )

```
  0   1   2   3
  +---+---+---+---+
0 | n | n | n | n |
  +---+---+---+---+
1 | n | y | n | y |
  +---+---+---+---+
2 | n | n | n | y |
  +---+---+---+---+
3 | y | n | n | y |
  +---+---+---+---+
```

## state number 
```
  0   1   2   3
  +---+---+---+---+
0 | 0 | 1 | 2 | 3 |
  +---+---+---+---+
1 | 4 | 5 | 6 | 7 |
  +---+---+---+---+
2 | 8 | 9 |10 |11 |
  +---+---+---+---+
3 |12 |13 |14 |15 |
  +---+---+---+---+
```

## rewards

```
  0   1   2   3
  +---+---+---+---+
0 | 0 | 0 | 0 | 0 |
  +---+---+---+---+
1 | 0 | 0 | 0 | 0 |
  +---+---+---+---+
2 | 0 | 0 | 0 | 0 |
  +---+---+---+---+
3 | 0 | 0 | 0 |+1 |
  +---+---+---+---+
```

In [4]:
print()
print("-->A 4x4 Grid World")
pprint(lake_environment.map)

print()
print("-->Number of states:", lake_environment.number_of_states)
print("-->And potential actions to take:", lake_environment.actions)

print()
print("-->With respective states numbered as follows:")
print()
for r in lake_environment.n_map:
    for c in r: 
        print('{:4d}'.format(c), end="")
    print()

print()
print("-->Total number of states:", lake_environment.number_of_states)
print()

print("--------------------")
print("Possible Conditions for each state")
print("--------------------")

for c in ['S', 'H', 'F', 'G']:
    print()
    print('state condition (char):', c)
    print("-->Reward:", lake_environment.reward[c])
    print("-->Is terminal?:", lake_environment.is_terminal[c])
    print("-->Icon:", lake_environment.icons[c])
    print()



-->A 4x4 Grid World
[['S', 'F', 'F', 'F'],
 ['F', 'H', 'F', 'H'],
 ['F', 'F', 'F', 'H'],
 ['H', 'F', 'F', 'G']]

-->Number of states: 16
-->And potential actions to take: ['up', 'down', 'left', 'right']

-->With respective states numbered as follows:

   0   1   2   3
   4   5   6   7
   8   9  10  11
  12  13  14  15

-->Total number of states: 16

--------------------
Possible Conditions for each state
--------------------

state condition (char): S
-->Reward: 0.0
-->Is terminal?: False
-->Icon: 👩


state condition (char): H
-->Reward: 0.0
-->Is terminal?: True
-->Icon: 💥


state condition (char): F
-->Reward: 0.0
-->Is terminal?: False
-->Icon: ▫️


state condition (char): G
-->Reward: 1.0
-->Is terminal?: True
-->Icon: 🎯



# Dynamic Programming

Dynamic programming assumes that the agent has full knowledge of the Markov Decision Process (MDP). We have the full knowledge of each one step dynamic of each state. 

For example you can run the following: 

```
possibilities = lake_environment.get_possibilities(state_number, action)
```

You get a `list` (aka `array`) of `FrozenLakeState` objects which each contains 
- probability of transitioning to this particular state `next_s` number given you came from a given state number and took the particular action 
- The corresponding reward `r` of landing to this next state `next_s`
- If this state is a terminal state 
- Among other useful formation

> DEFINITION: the **Transition Probability** of a state `s` (with corresponding current `state_number`) at time `t`, action `a` at timestep `t` and possible state `s'` (with corresponding `possible_state_number`) is the probability that the next state at timestep `t+1` is the possible_state `s'` given that at state `s` you do action `a`. 

```
*

transition_probability(s, a, s') = probability[state(t+1) = s'| state(t) = s, action(t) = a]

*
```

> The idea is that the surface is slippery, therefore the agent can slide to a location other than the one it wanted.

In [5]:
_ = lake_environment.get_possibilities(
    state_number=6, action='down', debug=True)

***
From state:  6  do action:  down !
***

# 1
--> next state:  10
--> reward: 0.0
--> probability:  0.3333333333333333
--> is terminal:  False


# 2
--> next state:  5
--> reward: 0.0
--> probability:  0.3333333333333333
--> is terminal:  True


# 3
--> next state:  7
--> reward: 0.0
--> probability:  0.3333333333333333
--> is terminal:  True



In [6]:
possibilities = lake_environment.get_possibilities(
    state_number=0, action='up', debug=False)

for i, state_info in enumerate(possibilities):
    
    print("--")
    print("#", i + 1)
    print("--")

    print(state_info)

--
# 1
--

*FrozenLakeState (TYPE) 
--> State Number: 0
--> Reward: 0.0
--> Not a terminal state. 
--> (Transition) Probability (given state_number and action): 0.3333333333333333
--> Icon: 👩
--> Character representation of state condition: 'S'
--> Location: (0,0) 


--
# 2
--

*FrozenLakeState (TYPE) 
--> State Number: 0
--> Reward: 0.0
--> Not a terminal state. 
--> (Transition) Probability (given state_number and action): 0.3333333333333333
--> Icon: 👩
--> Character representation of state condition: 'S'
--> Location: (0,0) 


--
# 3
--

*FrozenLakeState (TYPE) 
--> State Number: 1
--> Reward: 0.0
--> Not a terminal state. 
--> (Transition) Probability (given state_number and action): 0.3333333333333333
--> Icon: ▫️
--> Character representation of state condition: 'F'
--> Location: (0,1) 




In [7]:
possibilities = lake_environment.get_possibilities(
    state_number=14, action='right', debug=False)

print("You are in state number: 14, and you do action: right")
for i, state_info in enumerate(possibilities):
    
    print("--")
    print("#", i + 1)
    print("--")

    print("next_state: ", state_info.n)
    print("reward: ", state_info.reward)
    print("this is a terminal state?", state_info.is_terminal)
    print("probability of transitioning to this state")
    print("coming from state number 14 and going right: ", state_info.probability)

You are in state number: 14, and you do action: right
--
# 1
--
next_state:  15
reward:  1.0
this is a terminal state? True
probability of transitioning to this state
coming from state number 14 and going right:  0.3333333333333333
--
# 2
--
next_state:  10
reward:  0.0
this is a terminal state? False
probability of transitioning to this state
coming from state number 14 and going right:  0.3333333333333333
--
# 3
--
next_state:  14
reward:  0.0
this is a terminal state? False
probability of transitioning to this state
coming from state number 14 and going right:  0.3333333333333333


In [8]:
_ = lake_environment.get_possibilities(
    state_number=7, action='left', debug=True)

***
From state:  7  do action:  left !
***

# 1
--> next state:  7
--> reward: 0.0
--> probability:  1.0
--> is terminal:  True



# Policy Evaluation

In [9]:
def create_equiprobable_policy(env):

    # probability of taking an action given the state 
    p = 1.0 / len(env.actions)
    
    policy_for_any_state = {}
    policy = {}
    for action in env.actions:
        policy_for_any_state[action] = p
    
    for state_number in range(env.number_of_states):
        policy[state_number] = policy_for_any_state
    
    return policy

policy = create_equiprobable_policy(env = lake_environment)
pprint(policy)

{0: {'down': 0.25, 'left': 0.25, 'right': 0.25, 'up': 0.25},
 1: {'down': 0.25, 'left': 0.25, 'right': 0.25, 'up': 0.25},
 2: {'down': 0.25, 'left': 0.25, 'right': 0.25, 'up': 0.25},
 3: {'down': 0.25, 'left': 0.25, 'right': 0.25, 'up': 0.25},
 4: {'down': 0.25, 'left': 0.25, 'right': 0.25, 'up': 0.25},
 5: {'down': 0.25, 'left': 0.25, 'right': 0.25, 'up': 0.25},
 6: {'down': 0.25, 'left': 0.25, 'right': 0.25, 'up': 0.25},
 7: {'down': 0.25, 'left': 0.25, 'right': 0.25, 'up': 0.25},
 8: {'down': 0.25, 'left': 0.25, 'right': 0.25, 'up': 0.25},
 9: {'down': 0.25, 'left': 0.25, 'right': 0.25, 'up': 0.25},
 10: {'down': 0.25, 'left': 0.25, 'right': 0.25, 'up': 0.25},
 11: {'down': 0.25, 'left': 0.25, 'right': 0.25, 'up': 0.25},
 12: {'down': 0.25, 'left': 0.25, 'right': 0.25, 'up': 0.25},
 13: {'down': 0.25, 'left': 0.25, 'right': 0.25, 'up': 0.25},
 14: {'down': 0.25, 'left': 0.25, 'right': 0.25, 'up': 0.25},
 15: {'down': 0.25, 'left': 0.25, 'right': 0.25, 'up': 0.25}}


In [10]:
import numpy as np

def get_expected_return(V, state_number, action, env, gamma):
    expected_return = 0.0
    possibilities = env.get_possibilities(state_number, action)
    for state_info in possibilities: 
        r = state_info.reward
        n = state_info.n
        p = state_info.probability
        expected_return += (p * (r + gamma * V[n]))
    return expected_return 


def update_state_value(V, state_number, policy, env, gamma):
    new_v = 0.0
    for action, action_probability in policy[state_number].items():
        expected_return = get_expected_return(V, state_number, action, env, gamma)
        new_v += (action_probability * expected_return)
    return new_v
        
    
def policy_evaluation(env, policy, gamma=1, theta=1e-8):
    V = np.zeros(env.number_of_states)
    i = 0
    while True:
        delta = 0.0
        for state_number in range(env.number_of_states):
            old_v = V[state_number]
            new_v = update_state_value(V, state_number, policy, env, gamma)
            value_difference = np.abs(old_v - new_v)
            delta = max(delta, value_difference)
            V[state_number] = new_v
        i+=1
        print(i)
        print(V)
        if delta < theta: break
    return V

policy = create_equiprobable_policy(env = lake_environment)
V = policy_evaluation(lake_environment, policy)
pprint(V)

1
[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.25 0.  ]
2
[0.      0.      0.      0.      0.      0.      0.      0.      0.
 0.      0.0625  0.      0.      0.0625  0.34375 0.     ]
3
[0.         0.         0.         0.         0.         0.
 0.015625   0.         0.         0.03125    0.09765625 0.
 0.         0.109375   0.38769531 0.        ]
4
[0.         0.         0.00390625 0.00097656 0.         0.
 0.02539062 0.         0.0078125  0.05371094 0.11669922 0.
 0.         0.13769531 0.41052246 0.        ]
5
[0.         0.00097656 0.0078125  0.00244141 0.00195312 0.
 0.03112793 0.         0.01586914 0.06756592 0.12730408 0.
 0.         0.15394592 0.42294312 0.        ]
6
[0.00073242 0.00238037 0.01094055 0.00395584 0.00463867 0.
 0.03456116 0.         0.02201843 0.07581711 0.13333035 0.
 0.         0.16317654 0.4298625  0.        ]
7
[0.00212097 0.00386047 0.01332951 0.0053103  0.00719452 0.
 0.03666496 0.         0.02625751 0.0806911  0.13680464 0.
 0.  

In [17]:

[0.01393977, 0.01163091, 0.02095297, 0.01047648, 0.01624865, 0.        , 0.04075153, 0.        , 0.03480619, 0.08816993,
       0.14205316, 0.        , 0.        , 0.17582037, 0.43929118,
       0.        ] ==  [0.01393977, 0.01163091, 0.02095297, 0.01047648, 0.01624865, 0.,
 0.04075153, 0.,         0.03480619, 0.08816993, 0.14205316, 0.,
 0.,         0.17582037, 0.43929118, 0.        ]

True